In [ ]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

class Crawler:

    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls
        self.linksearch = {
            "word(count)":[],
            "URL":[]
        }
        
    def download_url(self, url):
        return requests.get(url).text

    def get_linked_urls(self, url, html):
        soup = BeautifulSoup(html, 'html.parser')
        for link in soup.find_all('a'):
            path = link.get('href')
            if path and path.startswith('/'):
                path = urljoin(url, path)
            yield path

    def add_url_to_visit(self, url):
        if url not in self.visited_urls and url not in self.urls_to_visit:
            if '#' not in url or '?' not in url or 'None' not in url:
                self.urls_to_visit.append(url)

    def crawl(self, url):
        html = self.download_url(url)
        self.search_title(html,url)
        self.search(html,url)
        for url in self.get_linked_urls(url, html):
            self.add_url_to_visit(url)


    def run(self):
#         i = 0
        while self.urls_to_visit:
            url = self.urls_to_visit.pop(0)
#             logging.info(f'Crawling: {url}')
            try:
#                 if i == 20:                    
#                     break
                self.crawl(url)
#                 i += 1
            except Exception:
                pass
#                 logging.exception(f'Failed to crawl: {url}')
            finally:
                self.visited_urls.append(url)

    def search(self,html,url):
        searched_word = "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย"
        soup = BeautifulSoup(html, 'html.parser')
        results = soup.body.find_all(string=re.compile('.*{0}.*'.format(searched_word)), recursive=True)
        print('Found the word "{0}" {1} times in {2}\n'.format(searched_word, len(results),url))
#         print(results)
    def search_title(self,html,url):
        searched_word = "Sono Bisque Doll wa Koi wo Suru"
        soup = BeautifulSoup(html, 'html.parser')
        results = soup.title.find_all(string=re.compile('.*{0}.*'.format(searched_word)), recursive=True)
        if  len(results) >= 1:
            self.linksearch["word(count)"].append(str(len(results)))
            self.linksearch["URL"].append(url)
#         df = pd.DataFrame(self.linksearch)
#         print(self.linksearch)
#         print(df)

        


if __name__ == '__main__':
    Crawler(urls=['https://myanimelist.net/anime/genre/48/Work_Life','https://anime-kimuchi.com/','https://anime-fast.online/']).run()



Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://myanimelist.net/anime/genre/48/Work_Life

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 1 times in https://anime-kimuchi.com/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://mxj.myanimelist.net/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://myanimelist.net/membership?_location=mal_h_u

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://myanimelist.net/login.php?from=%2Fanime%2Fgenre%2F48%2FWork_Life

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนท

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%ad%e0%b8%99%e0%b8%b4%e0%b9%80%e0%b8%a1%e0%b8%b0%e0%b8%a2%e0%b8%b9%e0%b8%a3%e0%b8%b4-yuri-anime-%e0%b8%8b%e0%b8%b1%e0%b8%9a%e0%b9%84%e0%b8%97%e0%b8%a2/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%ad%e0%b8%99%e0%b8%b4%e0%b9%80%e0%b8%a1%e0%b8%b0%e0%b8%81%e0%b8%b5%e0%b8%ac%e0%b8%b2/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%ad%e0%b8%99%e0%b8%b4%e0%b9%80%e0%b8%a1%e0%b8%b0%e0%b8%88%e0%b8%b5%e0%b8%99%e0%b8%8b%e0%b8%b1%e0%b8%9a%e0%b9%84%e0%b8%97%e0%b8%a2/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%ad%e0%b8%99%e0%b8%b4%e0%b9%

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%81%e0%b8%b2%e0%b8%a3%e0%b9%8c%e0%b8%95%e0%b8%b9%e0%b8%99%e0%b9%82%e0%b8%ab%e0%b8%94-%e0%b8%ad%e0%b8%99%e0%b8%b4%e0%b9%80%e0%b8%a1%e0%b8%b0%e0%b8%aa%e0%b8%a2%e0%b8%ad%e0%b8%87%e0%b8%82%e0%b8%a7/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%81%e0%b8%b2%e0%b8%a3%e0%b9%8c%e0%b8%95%e0%b8%b9%e0%b8%99%e0%b9%83%e0%b8%aa%e0%b9%88%e0%b9%81%e0%b8%a7%e0%b9%88%e0%b8%99-%e0%b8%81%e0%b8%b2%e0%b8%a3%e0%b9%8c%e0%b8%95%e0%b8%b9%e0%b8%99%e0%b8%9c/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/category/%e0%b8%ad%e0%b8%99%e0%b8%b4%e0%b9%80%e0%b8%a1%e0%b8%b0-supernatural/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไ

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 1 times in https://anime-master.net/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://hydra888.co

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://panama888.co

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://record.mytopaff.com/_Nb5-HdRK9V8lkhzSQlkHu2Nd7ZgqdRLk/1/?payload=anime-kimuchi_F30_ads

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://ufazeed.co

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://lockdown168.co

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://lotto4321.com

Foun

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/2019/10/03/tales-of-demons-and-gods-3d-%e0%b8%a0%e0%b8%b2%e0%b8%843-%e0%b8%95%e0%b8%ad%e0%b8%99%e0%b8%97%e0%b8%b5%e0%b9%88-81-120-%e0%b8%8b%e0%b8%b1%e0%b8%9a%e0%b9%84%e0%b8%97%e0%b8%a2/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/2021/04/11/86-eighty-six-86-%e0%b9%80%e0%b8%ad%e0%b8%97%e0%b8%95%e0%b8%b5%e0%b9%89%e0%b8%8b%e0%b8%b4%e0%b8%81%e0%b8%8b%e0%b9%8c-%e0%b8%95%e0%b8%ad%e0%b8%99%e0%b8%97%e0%b8%b5%e0%b9%88-%e0%b8%8b%e0%b8%b1%e0%b8%9a%e0%b9%84%e0%b8%97%e0%b8%a2/

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-kimuchi.com/2022/01/09/kaijin-kaihatsu-bu-no-kuroitsu-san-%e0%b8%95%e0%b8%ad%e0%b8%99%e0%b8%97%e0%b8%b5%e0%b9%88-%e0%b8%8b%e0%b8%b1%e0%b8%9a%e0%b9%84%e0%b8%97%e0%b8%a2/

Found the 

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2615/kaijin-kaihatsu-bu-no-kuroitsu-san-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2724/rymans-club-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2614/genjitsu-shugi-yuusha-no-oukoku-saikenki-2-ยุทธศาสตร์กู้ชาติของราชามือใหม่-ภาค2-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2511/lupin-iii-part-6-จอมโจรลูแปงที่สาม-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2613/akebi-chan-no-sailor-fuku-อะเคบิจังโนะเซเลอร์ฟุคุ-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru 

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2708/xing-chen-bian-4-stellar-transformation-season-4-การผันแปรของดวงดาว-ภาค4-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2621/futsal-boys-

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2728/long-shen-yanyi-dragons-disciple-ตำนานมังกรกับงู-ซับไทย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2620/sabiiro-no-armor-reimei

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://anime-fast.online/2619/baraou-no-souretsu-พันธนาการแห่งราชันย์กุหลาบ

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://otakumode.com/shop/611dc976892c240dccfc0895/Pop-Up-Parade-KonoSuba-Megumin-Swimsuit-Ver?utm_source=ad&utm_medium=affiliate&utm_campaign=mal_affiliate_2021

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://otakumode.com/shop/5d6479a5ec452c050ccc7b7b?utm_source=ad&utm_medium=affiliate&utm_campaign=mal_affiliate_2021

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://otakumode.com/shop/60d3d73bfa4a9a1562c655df?utm_source=ad&utm_medium=affiliate&utm_campaign=mal_affiliate_2021

Found the word "Sono Bisque Doll wa Koi wo Suru หนุ่มเย็บผ้ากับสาวนักคอสเพลย์ ตอนที่ 1-11 ซับไทย" 0 times in https://otakumode.com/shop/52ba4cca572781c718000074?utm_source=ad&utm_medium=affiliate&utm_campaign=mal_affiliate_2021

Found the word "Sono Bisque Doll wa Koi 

In [ ]:
data = {'word(count)': ['1'], 'URL': ['https://www.thsport.com/sitemap.xml']}
df = pd.DataFrame(data)
print(df)